In [6]:
import requests
import json
from tabulate import tabulate
# import coki.funcs as funcs
import funcs
import pandas as pd
from datetime import datetime
import os
import importlib

In [2]:
funcs.gen_table('GOGLD')

	 GOGLD - Cedear Google inc. - USD - 17:50:23
	 Last: 2.33 - High: 2.35 - Low: 2.2 - Volume: 15198
╭────────────────┬─────────────────┬────┬────────────────┬───────────────╮
│   Cant. compra │   Precio compra │    │   Precio venta │   Cant. venta │
├────────────────┼─────────────────┼────┼────────────────┼───────────────┤
│             50 │            2.20 │    │           2.33 │          4602 │
│              5 │            2.10 │    │           2.38 │           356 │
│             58 │            2.00 │    │           2.40 │           440 │
│             25 │            1.91 │    │           2.45 │           218 │
│                │                 │    │           2.46 │            50 │
╰────────────────┴─────────────────┴────┴────────────────┴───────────────╯


In [23]:
def gen_table_orders(ticker="ALL", filtro="cancelado"):
    orders = funcs.get_orders_link()

    tables = []
    for i in range(0, len(orders)):
        data = orders[i]

        if data["currency"] == "USD":
            ticker_name = data["ticker"] + "D"
        else:
            ticker_name = data["ticker"]
        
        if data["status"] == 'MARKET':
            ejecutadas = 0
        else:
            ejecutadas = data["result_quantity"]
        table = {
            "Hora": datetime.strptime(data["date"], "%Y-%m-%dT%H:%M:%S.%f%z")
            .time()
            .strftime("%H:%M"),
            "Ticker": ticker_name,
            "Estado": data["status"],
            "Operación": data["order_type"],
            "Precio": round(data["set_price"], 2),
            "Cantidad": int(round(data["set_quantity"], 0)),
            "Monto": round(data["set_amount"], 2),
            "Ejecutadas": ejecutadas,
            "code_op": data["order_id"],
        }
        tables.append(table)

    df = pd.DataFrame(tables)
    df = df.copy().sort_values(by=["Hora"], ascending=True).reset_index(drop=True)
    df["Op"] = df.index
    df = df.copy().set_index("Op")

    df["Estado"] = df["Estado"].replace(
        {
            "MARKET": "En mercado",
            "CANCELLED": "Cancelada",
            "REJECTED": "Rechazada",
            "EXECUTED": "Ejecutada",
            "PARTIALLY_EXECUTED": "Ejecutada parcialmente",
        }
    )

    if filtro == "cancelado":
        df = df[(df["Estado"] != "Cancelada") & (df["Estado"] != "Rechazada")].copy()
    elif filtro == "ejecutado":
        df = df[df["Estado"] == "Ejecutada"].copy()
    elif filtro == "mercado":
        df = df[
            (df["Estado"] == "Ejecutada parcialmente") | (df["Estado"] == "En mercado")
        ].copy()

    df["Operación"] = df["Operación"].replace(
        {
            "BUY": "Compra",
            "SELL": "Venta",
        }
    )

    if ticker != "ALL":
        df = df[df["Ticker"] == ticker].copy()

    df = df.sort_values(by=["Ticker", "Hora"], ascending=False)

    dict_operaciones = df["code_op"].to_dict()

    time_str = datetime.now().strftime("%H:%M:%S")

    mail = funcs.read_credentials()[0]

    print(
        "\n \t \t \t",
        "Actualizado a:", 
        time_str,
        "-",
        " Cuenta:",
        mail,
        "\n"
    )
    
    print(
        tabulate(
            df[df.columns[:-1]],
            headers="keys",
            tablefmt="rounded_outline",
            floatfmt=".2f",
            stralign="center",
        )
    )
    return dict_operaciones


In [24]:
data = gen_table_orders()


 	 	 	 Actualizado a: 18:23:00 -  Cuenta: nicolaspcastro@gmail.com 

╭──────┬────────┬──────────┬───────────┬─────────────┬──────────┬────────────┬─────────┬──────────────╮
│   Op │  Hora  │  Ticker  │  Estado   │  Operación  │   Precio │   Cantidad │   Monto │   Ejecutadas │
├──────┼────────┼──────────┼───────────┼─────────────┼──────────┼────────────┼─────────┼──────────────┤
│   15 │ 12:14  │  TSLAD   │ Ejecutada │    Venta    │    16.90 │         10 │  169.00 │           10 │
│   12 │ 11:43  │  TSLAD   │ Ejecutada │   Compra    │    16.65 │         10 │  166.50 │           10 │
│   34 │ 15:52  │  METAD   │ Ejecutada │    Venta    │    14.05 │         15 │  210.75 │           15 │
│   28 │ 14:32  │  METAD   │ Ejecutada │   Compra    │    13.95 │         15 │  209.25 │           15 │
│   33 │ 15:52  │  MELID   │ Ejecutada │    Venta    │    26.50 │         15 │  397.50 │           15 │
│   30 │ 15:08  │  MELID   │ Ejecutada │   Compra    │    26.40 │         15 │  396.00 │          

In [12]:
link = "https://api.cocos.capital/api/v1/users/me"
token = funcs.token_promise()
j = funcs.request_info(link, token, info_type='account')
j

{'id': '31e77750-e0d8-4d34-a8a7-6999525f7ea3',
 'tier': 'FREE',
 'first_name': 'NICOLAS PAULINO',
 'middle_name': None,
 'last_name': 'CASTRO',
 'cuit': '20367198932',
 'dni': '36719893',
 'phone': '+5492966595871',
 'email': 'nicolaspcastro@gmail.com',
 'birth_date': '1992-10-14',
 'street_name': 'MONSEÑOR FAGNANO',
 'street_number': '252',
 'floor': '',
 'apartment': '',
 'state_name': 'SANTA CRUZ',
 'zip_code': '9400',
 'bot_on': True,
 'last_bot_sign_in': None,
 'dark_theme': False,
 'is_active': True,
 'id_accounts': [55743],
 'status_onboarding': 'true',
 'favoritesLists': [{'id_list': 'fbd07022-a3b2-48c9-83a3-b8e16ec22844',
   'name': 'Favoritos'}]}